In [ ]:
import numpy as np
import nltk
import random
import string
from nltk.corpus import stopwords
import urllib.request
from bs4 import BeautifulSoup
import speech_recognition as sr
import pyttsx3
from gtts import gTTS
from playsound import playsound
from pandas import read_csv
import csv

In [ ]:
raw_doc_read=read_csv('train.csv')

txt_file='data.txt'
csv_file='train.csv'

with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r",errors='ignore') as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

doc_read=open('data.txt','r')
raw_doc_read=doc_read.read()

In [ ]:
lang='en'
tld='co.uk'

In [ ]:
# raw_doc_read=raw_doc_read.lower()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

In [ ]:
sent_tokens=nltk.sent_tokenize(raw_doc_read)
word_tokens=nltk.word_tokenize(raw_doc_read)

In [ ]:
lemmer=nltk.stem.WordNetLemmatizer()

def lemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

no_punc_dict=dict((ord(punct), None) for punct in string.punctuation)

def lemNormalize(txt):
    return lemTokens(nltk.word_tokenize(txt.lower().translate(no_punc_dict)))

In [ ]:
greetings=('hi','hello','hey')
greeting_resp=('Hello!','Hi!','Hey!','Hey there!')

def greet(sentence):
    for word in sentence.lower().split():
        if word in greetings:
            return random.choice(greeting_resp)+' I am Rikuduo.'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def botResponse(user_input):
    robo_response=''
    tfid_vec=TfidfVectorizer(tokenizer=lemNormalize,stop_words=stopwords.words('english'))
    tfid_fit=tfid_vec.fit_transform(sent_tokens)
    values=cosine_similarity(tfid_fit[-1],tfid_fit)

    index=values.argsort()[0][-2]
    flat=values.flatten()
    flat.sort()
    req_tfidfit=flat[-2]
    if(req_tfidfit==0):
        robo_response+='Sorry. I am unable to understand you.'
        return robo_response
    else:
        robo_response+=sent_tokens[index]
        return robo_response

In [ ]:
recognizer=sr.Recognizer()
def SpeakText(command):
    engine=pyttsx3.init()
    engine.say(command)
    engine.runAndWait()

In [ ]:
def botVoice(bot_resp):
    audio_obj=gTTS(text=bot_resp,lang=lang,slow=False,tld=tld)
    audio_obj.save('response.mp3')
    playsound('D:/VSCode(Chatbot)/response.mp3')

In [ ]:
flag=True
print('Hello! This is Rikuduo Bot. To start, greet me and to end, type bye!')
botVoice('Hello! This is Rikuduo Bot. To start, greet me and to end, type bye!')

In [ ]:
while(flag):
    print('Start speaking: ')
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source,duration=0.2)
        audio=recognizer.listen(source)

        user_input=recognizer.recognize_google(audio)
        user_input=user_input.lower()
    print("User: "+user_input)
    if(user_input!='bye'):
        if(user_input=='thank you' or user_input=='thanks'):
            print('Rikuduo: You are welcome!')
            botVoice('You are welcome')
        else:
            if(greet(user_input)!=None):
                bot_resp=greet(user_input)
                print('Rikuduo: '+bot_resp)
                botVoice(bot_resp)
            else:
                sent_tokens.append(user_input)
                word_tokens=word_tokens+nltk.word_tokenize(user_input)
                final_words=list(set(word_tokens))
                print('Rikuduo: ',end='')
                bot_resp=botResponse(user_input)
                print(bot_resp)
                botVoice(bot_resp)

                sent_tokens.remove(user_input)
    else:
        flag=False
        print('Rikuduo: Goodbye!')